---
title: Graphs
order: 5
---

This post implements (and improves) code in chapter 14 from the textbook Introduction to Computation and Programming Using Python, With Application to Computational Modeling and Understanding Data, third edition, by John V. Guttag, 2020. 

In [2]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Tuple

In [3]:
@dataclass(frozen=True)
class Node:
    """Basic immutable graph node."""

    name: str

    def __str__(self) -> str:
        return self.name

Designing the inheritance pattern is difficult to me here. The substitution principle: A code that works on a superclass must work on a subclass. But if a code works on bidirectional edges (perhaps exploiting symmetry) it may not work on unidirectional edges (where the symmetry doesn't exist). So, if anything, bidirectional edge must inherit from unidirectional edge. But that doesn't work either, because unidirectional edge logically must have the sources and destinations, but bidirectional edge cannot have them. Which node should be returned by a bidirectional edge's "source" property? There is no clear answer, so I conclude bidirectional edge cannot inherit from unidirectional. Together these two facts mean neither can inherit from the other.

One solution would be to completely discard bidirectional edges and only work with directional ones - after all, we can build undirected graph with directed edges. But that would mean, while building an undirected graph, that we treat unidirectional edges as bidirectional, which is weird.

BaseEdge is a abstract base class for edges. An edge has two nodes, node1 and node2, can return them in a tuple (.nodes) and can be represented by a string where names of both nodes are separated by a separator. Also is hashable, which means it implements \_\_eq\_\_ and \_\_hash\_\_ methods.
_separator, \_\_eq\_\_ and \_\_hash\_\_ are abstract methods. Their implementation depends on whether the edge is bidirectional or unidirectional.

@ property means accessing `.nodes` on in instance will call this *method*, which works as a getter. It also blocks setting the attribute (via `.nodes = something`). But it doesn't block its mutation, were it e.g. a list.

In [4]:
@dataclass(frozen=True)
class BaseEdge(ABC):
    """Abstract base class for graph edge."""

    _node1: Node
    _node2: Node

    @property
    def nodes(self) -> Tuple[Node, Node]:
        return self._node1, self._node2

    @property
    @abstractmethod
    def _separator(self) -> str:
        pass

    def __str__(self) -> str:
        node1, node2 = self.nodes
        return f"{node1} {self._separator} {node2}"


@dataclass(frozen=True)
class Edge(BaseEdge):
    """Bidirectional unweighted graph edge."""

    def __init__(self, node1: Node, node2: Node):
        # Improves signature of the constructor
        # Canonical ordering for dataclass's __eq__ and __hash__
        # to work correctly and for better str representation
        if node1.name > node2.name:
            BaseEdge.__init__(self, node2, node1)
        else:
            BaseEdge.__init__(self, node1, node2)

    @property
    def _separator(self) -> str:
        return "<---->"


@dataclass(frozen=True)
class DirectedEdge(BaseEdge):
    """Unidirectional unweighted graph edge."""

    def __init__(self, src: Node, dest: Node):
        # Improves signature of the constructor
        # Invariant: src=node1 and dest=node2
        BaseEdge.__init__(self, src, dest)

    @property
    def src(self) -> Node:
        return self._node1

    @property
    def dest(self) -> Node:
        return self._node2

    @property
    def _separator(self) -> str:
        return "----->"


@dataclass(frozen=True)
class WeightedEdgeMixin:
    weight: int | float


@dataclass(frozen=True)
class WeightedEdge(Edge, WeightedEdgeMixin):
    """Weighted bidirectional graph edge."""

    def __init__(self, node1: Node, node2: Node, weight: int | float):
        Edge.__init__(self, node1, node2)
        WeightedEdgeMixin.__init__(self, weight)

    @property
    def _separator(self) -> str:
        return f"<--({self.weight})-->"


@dataclass(frozen=True)
class WeightedDirectedEdge(DirectedEdge, WeightedEdgeMixin):
    """Weighted unidirectional graph edge."""

    def __init__(self, src: Node, dest: Node, weight: int | float):
        DirectedEdge.__init__(self, src, dest)
        WeightedEdgeMixin.__init__(self, weight)

    @property
    def _separator(self) -> str:
        return f"---({self.weight})-->"

### Testing of Nodes and Edges

In [5]:
node1 = Node("1")
node2 = Node("2")

# BaseEdge()  # TypeError: Can't instantiate abstract class BaseEdge without an implementation for abstract method '_separator'

e12 = Edge(node1, node2)
we12_314 = WeightedEdge(node1, node2, 3.14)
dir12 = DirectedEdge(node1, node2)
wdir12_314 = WeightedDirectedEdge(node1, node2, 3.14)

e21 = Edge(node2, node1)
we21_314 = WeightedEdge(node2, node1, 3.14)
dir21 = DirectedEdge(node2, node1)
wdir21_314 = WeightedDirectedEdge(node2, node1, 3.14)

we12_3 = WeightedEdge(node1, node2, 3)
wdir12_3 = WeightedDirectedEdge(node1, node2, 3)

In [6]:
print(f"{e12} equals {e21} is {e12 == e21}")
print(f"{we12_314} equals {we21_314} is {we12_314 == we21_314}")
print(f"{dir12} equals {dir21} is {dir12 == dir21}")
print(f"{wdir12_314} equals {wdir21_314} is {wdir12_314 == wdir21_314}")
print(f"{we12_314} equals {we12_3} is {we12_314 == we12_3}")
print(f"{wdir12_314} equals {wdir12_3} is {wdir12_314 == wdir12_3}")

1 <----> 2 equals 1 <----> 2 is True
1 <--(3.14)--> 2 equals 1 <--(3.14)--> 2 is True
1 -----> 2 equals 2 -----> 1 is False
1 ---(3.14)--> 2 equals 2 ---(3.14)--> 1 is False
1 <--(3.14)--> 2 equals 1 <--(3)--> 2 is False
1 ---(3.14)--> 2 equals 1 ---(3)--> 2 is False


In [ ]:
print({e12, e21})  # exactly what we wanted because they are the same edge!
print({we12_314, we21_314})
print({node1, Node("1"), node2})

{Edge(_node1=Node(name='1'), _node2=Node(name='2'))}
{WeightedEdge(weight=3.14, _node1=Node(name='1'), _node2=Node(name='2'))}
{Node(name='2'), Node(name='1')}


In [9]:
class C:
    pass


{C(), C()}

{<__main__.C at 0x1fdab7befd0>, <__main__.C at 0x1fdab7eea50>}